# TP2

In [ ]:
import re
import requests
import io
import itertools

from IPython.display import clear_output
from bs4 import BeautifulSoup

# Déclarations des fonctions

In [ ]:
# Enlever une certains caractères dans une chaine de caractère
def rchop(thestring, ending):
    if thestring.endswith(ending):
        return thestring[:-len(ending)]
    return thestring

# Fonction d'affichage
def print_rules(bla):
    for rule in bla:
        print(str(rule).replace(', ', '==>'))

In [ ]:
# Déclarations des variables
nature = {'Verbe': 'V', 'Nom': 'N', 'Adjectif': 'ADJ', 'Nom propre': 'NP'}
rules = []
result = {}

# Lecture du mot
print(f'Entrer le mot : ')
testWord = input()

# Effacer les outputs
clear_output(wait = False)

# Lecture de la nature du mot
print(f'Veuillez choisir la nature du mot dans la liste ci-dessous : ')
for (key, value) in nature.items():
    print(f'\t{value} pour {key}')
natureWord = input()

# Effacer les outputs
clear_output(wait = False)
while natureWord not in nature.values():
    print(f'Veuillez choisir la nature du mot dans la liste ci-dessous : ')
    for (key, value) in nature.items():
        print(f'\t{value} pour {key}')
    natureWord = input()

    # Effacer les outputs
    clear_output(wait = False)

# Ouverture du fichier
file = open("rules.txt", "r")

# Lecture du fichier & enregistrement des règles
if file.mode == 'r':
    lines = file.readlines()
    for line in lines:
        rules.append(line.rstrip("\n").split("==>"))

# Fermeture du fichier
file.close()

# Génération des mots et leur nature
for rule in rules:
    flag = False
    ruleG = rule[0].split(":")
    ruleD = rule[1].split(":")
    match = re.match(".*" + ruleG[1], testWord)
    if match and ruleG[0] == natureWord:
        if ruleG[1] != "":
            generatedWord = rchop(testWord, ruleG[1]) + ruleD[1]
        else:
            generatedWord = testWord + ruleD[1]
        result.setdefault(generatedWord, [])
        if(len(result[generatedWord]) != 0):
            for n in result[generatedWord]:
                if(n == ruleD[0]):
                    flag = True
            if(not flag):
                result[generatedWord].append(ruleD[0])
        else:
            result[generatedWord].append(ruleD[0])

if(result):
    print(f'\nLes dérivations du mot : {testWord}')
    for n in result.items():
        temp = str(n).replace(", [", " de nature ").replace("[", "").replace("]", "").replace("'", "").replace("(", "").replace(")", "")
        print(f'\t{temp}')
else:
    print('Aucun mot n\'est valide')

# Affichage des règles
#print_rules(rules)

In [ ]:
finalD = {}

for word in result:
    # Envoie de la requete à jdm
    url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(word) + "&rel="
    
    # Reponse de la requete
    response = requests.get(url)
    
    # bs4
    soup = BeautifulSoup(response.text, features = 'lxml')
    
    # Chercher la balise "jdm-warning"
    mydivs = soup.findAll("div", {"class": "jdm-warning"})
    
    if(not mydivs): # if jdm-warning existe (mot invalide)
        finalD.setdefault(word, [])
        if(len(finalD[word]) != 0):
            for n in finalD[word]:
                if(n == result[word]):
                    flag = True
            if(not flag):
                finalD[word].append(result[word])
        else:
            finalD[word].append(result[word])
    
if(finalD):
    print('Les mots existants dans jeux de mots :')
    for n in finalD.items():
        temp = str(n).replace(", [", " de nature ").replace("[", "").replace("]", "").replace("'", "").replace("(", "").replace(")", "")
        print(f'\t{temp}')
else:
    print('Aucun mot n\'est valide')

In [ ]:
finalDD = {}

for word in finalD:
    a1 = ''
    # Envoie de la requete à jdm
    url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(testWord) + "&rel=0"
    
    # Reponse de la requete
    response = requests.get(url)

    # bs4
    soup = BeautifulSoup(response.text, features = 'lxml')

    # Chercher la balise "CODE"
    mycode = soup.find('code')

    s = io.StringIO(str(mycode))
    for line in s:
        line = rchop(line, '\n')
        
        matchTestWord = re.match('e;.*;\'' + word + '\';.*;.*', line)
        if(matchTestWord):
            lineTestWord = line
            lineTestWordTemp = lineTestWord.split(';')
            a1 = lineTestWordTemp[1]
    
        if(a1):
            finalDD.setdefault(word, [])
            if(len(finalDD[word]) != 0):
                for n in finalDD[word]:
                    if(n == result[word]):
                        flag = True
                if(not flag):
                    finalDD[word].append(result[word])
            else:
                finalDD[word].append(result[word])
    
if(finalDD):
    print(f'Les mots existants dans jeux de mots qui sont en relation (r0) avec le mot : {testWord} :')
    for n in finalDD.items():
        temp = str(n).replace(", [", " de nature ").replace("[", "").replace("]", "").replace("'", "").replace("(", "").replace(")", "")
        print(f'\t{temp}')
else:
    print('Aucun mot n\'est valide')

In [ ]:
relation_map = {'AGENT': '13', 'LIEU' : '15', 'N': '4', 'V': '4', 'ADJ': '4', 'ADV': '4'}
type_map = {'N': 'Nom:', 'V' : 'Ver:', 'ADJ' : 'Adj:'}
taliD = {}

def blabla(word, line):
    matchTestWord = re.match('e.*;.*;\'' + word + '\';.*;.*', line)
    if(matchTestWord):
        lineTestWord = line
        print(f'ICI 1 : {lineTestWord}')

for f in finalDD:    
    for k in itertools.chain.from_iterable(finalDD[f]):
        a1 = ''
        a2 = ''
        flag = False
        # Envoie de la requete à jdm
        if(int(relation_map[k]) == 4):
            url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(f) + "&rel=" + str(relation_map[k])

            # Reponse de la requete
            response = requests.get(url)

            # bs4
            soup = BeautifulSoup(response.text, features = 'lxml')
            
            # Chercher la balise "CODE"
            mycode = soup.find('code')

            s = io.StringIO(str(mycode))
            for line in s:
                line = rchop(line, '\n')
            
                matchTestWord = re.match('e;.*;\'' + f + '\';.*;.*', line)
                if(matchTestWord):
                    lineTestWord = line
                    lineTestWordTemp = lineTestWord.split(';')
                    a1 = lineTestWordTemp[1]

                matchType = re.match('e;.*;\'' + str(type_map[k]) + '\';.*;.*', line)
                if(matchType):
                    lineType = line
                    lineTypeTemp = lineType.split(';')
                    a2 = lineTypeTemp[1]

                if(a2):
                    matchRule = re.match('r;.*;' + a1 + ';' + a2 + ';4;.*', line)
                    if(matchRule):
                        lineRule = line
                        lineRuleTemp = line.split(';')
                        a3 = int(lineRuleTemp[5])
                        if(a3 > 0):
                            taliD.setdefault(f, [])
                            if(len(taliD[f]) != 0):
                                for n in taliD[f]:
                                    if(n == k):
                                        flag = True
                                if(not flag):
                                    taliD[f].append(k)
                            else:
                                taliD[f].append(k)
        else:
            url = "http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel=" + str(testWord) + "&rel=" + str(relation_map[k])

            # Reponse de la requete
            response = requests.get(url)

            # bs4
            soup = BeautifulSoup(response.text, features = 'lxml')

            # Chercher la balise "CODE"
            mycode = soup.find('code')

            s = io.StringIO(str(mycode))
            for line in s:
                line = rchop(line, '\n')

                matchTestWord = re.match('e;.*;\'' + testWord + '\';.*;.*', line)
                if(matchTestWord):
                    lineTestWord = line
                    lineTestWordTemp = lineTestWord.split(';')
                    a1 = lineTestWordTemp[1]

                matchWord = re.match('e;.*;\'' + f + '\';.*;.*', line)
                if(matchWord):
                    lineWord = line
                    lineWordTemp = lineWord.split(';')
                    a2 = lineWordTemp[1]

                if(int(relation_map[k]) == 13):
                    matchRule = re.match('r;.*;' + a1 + ';' + a2 + ';13;.*', line)
                    if(matchRule):
                        lineRule = line
                        lineRuleTemp = line.split(';')
                        a3 = int(lineRuleTemp[5])
                        if(a3 > 0):
                            taliD.setdefault(f, [])
                            if(len(taliD[f]) != 0):
                                for n in taliD[f]:
                                    if(n == k):
                                        flag = True
                                if(not flag):
                                    taliD[f].append(k)
                            else:
                                taliD[f].append(k)

                if(int(relation_map[k]) == 15):
                    matchRule = re.match('r;.*;' + a1 + ';' + a2 + ';15;.*', line)
                    if(matchRule):
                        lineRule = line
                        lineRuleTemp = line.split(';')
                        a3 = int(lineRuleTemp[5])
                        if(a3 > 0):
                            taliD.setdefault(f, [])
                            if(len(taliD[f]) != 0):
                                for n in taliD[f]:
                                    if(n == k):
                                        flag = True
                                if(not flag):
                                    taliD[f].append(k)
                            else:
                                taliD[f].append(k)

if(taliD):
    print('Les mots finaux sont :')
    for n in taliD.items():
        temp = str(n).replace(", [", " de nature ").replace("[", "").replace("]", "").replace("'", "").replace("(", "").replace(")", "")
        print(f'\t{temp}')
else:
    print('Aucun mot n\'est valide')